# Key-Value RDD, Transformations & Actions

### 1. Transformations 
- groupByKey
- reduceByKey
- mapValues
- keys
- join(+ leftOuterJoin, rightOuterJoin)

### 2. Actions
- countByKey

##### 보충설명 ChatGPT
1. Transformation:
    - Transformation은 데이터셋을 기반으로 새로운 데이터셋을 생성하는 연산
    - Transformation은 Lazy Evaluation(지연 평가)을 따릅니다. 즉, Transformation 연산이 호출되었다고 해서 바로 실행되지는 않습니다. 대신 Spark는 이후에 Action이 호출될 떄까지 연산을 미루고 실행 계획을 최적화합니다.
    - Transformation의 예로는 'map()', 'filter()', 'groupByKey()' 등이 있습니다.
    - Transformation은 일반적으로 RDD(Resilient Distributed Dataset)이나 DataFrame 형태의 불변성(Immutable) 데이터 구조에 적용됩니다.

2. Action:
    - Action은 실제로 계산을 수행하고 결과를 반환하는 연산입니다.
    - Action 연산이 호출되면, Spark는 앞서 정의한 Transformation의 실행 계획을 수행하고 최종 결과를 생성합니다.
    - 결과는 드라이버 프로그램에 반환되거나 외부 데이터 저장소에 저장될 수 있습니다.
    - Action의 예로는 'count()', 'collect()', 'take()', 'reduce'가 있습니다.

In [1]:
from operator import add
from pyspark import SparkConf, SparkContext

import warnings 
warnings.simplefilter(action='ignore')

conf = SparkConf().setMaster('local').setAppName('category-review-average')
sc = SparkContext(conf=conf)

23/03/30 22:56:44 WARN Utils: Your hostname, Keemyoui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.79 instead (on interface en0)
23/03/30 22:56:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/30 22:56:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/30 22:56:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [39]:
### 예시
rdd = sc.parallelize([1,2,3,4,5]) # RDD 생성
filtered_rdd = rdd.filter(lambda x: x % 2 == 0) # Transformation: 짝수만 필터링하는 연산
result = filtered_rdd.collect() # Action 결과를 수집하여 드라이버 프로개름에 반환
result

[2, 4]

23/03/31 10:59:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3224462 ms exceeds timeout 120000 ms
23/03/31 10:59:08 WARN SparkContext: Killing executors is not supported by current scheduler.


# 1. Transformation

### 01. GroupByKey

##### 01-1. 강의 예제
- groupbBy: 주어지는 함수를 기준으로 Group
- groupByKey: 주어지는 Key를 기준으로 Group

In [48]:
# 1. GrooupBy: 사용자가 지정하는 함수를 기준으로 그룹핑
### 예시) 2로 나누었을 때 몫을 기준으로 그룹핑
rdd = sc.parallelize([1,1,2,3,5,8])
result = rdd.groupBy(lambda x: x % 2).collect()
result1 = sorted([(x, sorted(y)) for (x,y) in result])
print('result1: ',result1)

# 2. groupByKey: 주어진 Key를 기준으로 그룹핑
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)]) 
result2 = sorted(rdd.groupByKey().mapValues(len).collect())
print('result2: ', result2)

### 키-값 쌍 데이터를 포함하는 RDD를 생성합니다. => ('a',1), ('b',1), ('a',1)
### rdd.groupByKey(): 키를 기준으로 데이터를 그룹화합니다.=> ('a', [1,1]), ('b', [1])
### mapValues(len): 키별 원소 개수를 계산합니다.(RDD) => ('a', 2), ('b',1) 
### .collect(): RDD의 모든 원소를 드라이버 프로그램에 수집합니다.(리스트) => [('a',2), ('b',1)]

result1:  [(0, [2, 8]), (1, [1, 1, 3, 5])]
result2:  [('a', 2), ('b', 1)]


In [49]:
# 3. mapValues(list)
rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)]) 
result = sorted(rdd.groupByKey().mapValues(list).collect())
print('result: ', result)

### rdd = sc.parallelize([('a', 1), ('b', 1), ('a', 1)]): 
##### 키-값 쌍 데이터를 포함하는 RDD를 생성합니다. => ('a', 1), ('b', 1), ('a', 1).

### rdd.groupByKey()
##### groupByKey() 키를 기준으로 데이터를 그룹화합니다.(RDD) => ('a', <iterable object>), ('b', <iterable object>).
##### 여기서 <iterable object>는 각 키에 대한 값들을 포함하는 이터러블 객체입니다.

### .mapValues(list)
##### groupByKey()로 그룹화된 데이터에 대해, list 함수를 각 그룹의 값에 적용하여 이터러블 객체를 리스트로 변환합니다. 
##### 이 연산의 결과는 다음과 같은 데이터를 포함하는 RDD입니다: ('a', [1, 1]), ('b', [1]).

result:  [('a', [1, 1]), ('b', [1])]


In [5]:
# 4. 첫글자를 따서 그룹핑하기: groupBy 
grouped = sc.parallelize([
    "C", "C++", "Python", "Java", "C#"
]).groupBy(lambda x: x[0]).collect() # 각프로그램의 첫글자를 기준으로 그룹핑합니다. 

for k, v in grouped:
    print(k, list(v))       
    

C ['C', 'C++', 'C#']
P ['Python']
J ['Java']


In [56]:
# groupByKey예제

#### 준비하기 ####
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH",7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)], 3) 

y = x.groupByKey() # 아무것도 실행되지 않음. RDD를 만들겠다로 끝나

##### 문제풀기 (1) #####
print(y.getNumPartitions()) # y값이 갖는 파티션이 몇개인지, 디폴트로 3
# 3

#### 문제풀기 (2) #####
y = x.groupByKey(2) # 파티션이 몇개인지를 정해주게 된다.ㅠㅠ
print(y.getNumPartitions())

 
for t in y.collect():
    print(t[0], list(t[1]))    

3
2
MATH [7, 2, 8, 3]
ENGLISH [7, 4, 9, 4]
SCIENCE [7, 6, 9, 5]


##### 01-2. ChatGPT 예제

In [ ]:
# 간단한 예제 1

# 초기 데이터 생성
data = [("A", 1), ("B", 2), ("A", 3), ("B", 4), ("A", 5), ("B", 6)]

# RDD로 변환
rdd = sc.parallelize(data)

# groupByKey() 함수를 사용하여 키를 기준으로 데이터 그룹화
grouped_rdd = rdd.groupBykey()

# 그룹화된 데이터를 출력하기 쉬운 형식으로 변환
result = grouped_rdd.map(lambda x: (x[0], list(x1)))

In [68]:
# 복잡한 예제

#sc = SparkContext("local", "GroupByKey Complex Example")

# 초기 데이터 생성
data = [("apple", 3), ("banana", 5), ("orange", 2),
        ("apple", 4), ("banana", 7), ("orange", 6),
        ("apple", 8), ("banana", 4), ("orange", 9)]

# RDD로 변환
rdd = sc.parallelize(data)

# groupByKey() 함수를 사용하여 키를 기준으로 데이터 그룹화
grouped_rdd = rdd.groupByKey()

# 각 과일 그룹의 총 개수와 평균을 계산
result = grouped_rdd.map(lambda x: (x[0], sum(x[1]), sum(x[1]) / len(x[1]))).collect()
print(result)

[('apple', 15, 5.0), ('banana', 16, 5.333333333333333), ('orange', 17, 5.666666666666667)]


# 02. reduceByKey
- reduce: 주어지는 함수를 기준으로 요소들을 합침(action)
- reduceByKey: Key를 기준으로 그룹을 만들고 합침(trans)
    - 결과값으로 나온 RDD가 파티션을 유지함으로 트랜스포메이션
    
- 개념적으로는 groupByKey + Reduction, 그러나 훨씬 빠르다.

### 02-1. 강의 예제

In [73]:
# 예제 1. reduce(add) RDD에 있는 것들을 합침
# reduce는 action이었죠(reduceByKey는 transformation입니다.)
result1 = sc.parallelize([1,2,3,4,5]).reduce(add) 
print('reduce:', result1)

# 예제 2. reduceByKey, 키를 기준으로 함수를 수행한다.
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
result2 = sorted(rdd.reduceByKey(add).collect()) 
print('reduceByKey:', result2)

reduce: 15
reduceByKey: [('a', 2), ('b', 1)]


In [9]:
# 예제 3. 키값들을 기준으로 벨류를 더한다.ㅠ
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7), 
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)], 3) 

x.reduceByKey(lambda a,b: a+b).collect()

[('MATH', 20), ('ENGLISH', 24), ('SCIENCE', 27)]

### 02-2. Chat GPT 예제

In [74]:
# 예제1. 단어 빈도수 계산
data = ['apple', 'banana', 'apple', 'orange', 'banana', 'apple']
rdd = sc.parallelize(data)

# 단어를 (단어, 1) 형태의 키-값 쌍으로 변환
pair_rdd = rdd.map(lambda x: (x,1)) 

# 동일한 단어를 기준으로 값을 합산하여 빈도수 계산 
word_count = pair_rdd.reduceByKey(lambda a,b: a+b)

# 결과 출력
for word, count in word_count.collect():
    print(f'{word}: {count}')

apple: 3
banana: 2
orange: 1


In [85]:
# 예제2. 평균 온도 계산
data = [
    ('Seattle', 50),
    ('San FranCisco', 60),
    ('Seattle', 55),
    ('New York', 30), 
    ('San FranCisco', 58),
    ('New York', 25)
]
rdd = sc.parallelize(data)

# 도시 이름을 키로, (온도, 1) 형태의 값으로 변환 => data의 개수가 줄어들지 않는다.
pair_rdd = rdd.map(lambda x: (x[0], (x[1], 1)))

# 도시별 온도 합계 및 개수 계산 => 키값(도시명)을 기준으로 온도 더하기, 개수 더하기
sum_count_rdd = pair_rdd.reduceByKey(lambda a, b: (a[0] + b[0], a[1]+ b[1]))

# 도시별 평균 온도 계산
avg_temp = sum_count_rdd.mapValues(lambda x: x[0]/x[1])

# 결과 출력
for city, temp in avg_temp.collect():
    print(f"{city}: {temp:.1f}")

Seattle: 52.5
San FranCisco: 59.0
New York: 27.5


# 03. MapValues
- 함수를 Value에게만 적용
- 파티션과 키는 그대로 유지

### 03-1 강의안

In [86]:
# 예제 1번
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])]) 
def f(x): return len(x) #적용시킬 함수를 만들고
result = x.mapValues(f).collect()
print('result1:', result)

result1: [('a', 3), ('b', 1)]


In [14]:
x = sc.parallelize([("a", ["apple", "banana", "lemon"]), ("b", ["grapes"])])
def f(x): return len(x)
x.mapValues(f).collect()

[('a', 3), ('b', 1)]

### 03-2 ChatGPT

In [ ]:
# 에제 1번: 간단한거


# 05. CountByKey
- 각 키가 가진 요소들을 센다
- 위에 MapValues를 

In [15]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.countByKey().items())

[('a', 2), ('b', 1)]

# 06. Keys()
- Transformation
- 모든 Key를 가진 RDD를 생성

In [17]:
m = sc.parallelize([(1,2), (3,4)]).keys()
m.collect()

[1, 3]

In [19]:
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7), 
    ("SCIENC", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4), 
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)], 3) 

x.keys().distinct().count()

4

# 07. Joins()
- Transformation
- 여러개의 RDD를 합치는데 사용
- 대표적으로 두 가지의 Join 방식이 존재 
    - inner Join(join)
    - Outer Join(left outer, right outer)


### 001. Inner Join vs Outer Join

In [25]:
rdd1 = sc.parallelize([('foo', 1), ('bar', 2), ('baz', 3)])
rdd2 = sc.parallelize([('foo', 4), ('bar', 5), ('bar', 6), ('zoo', 1)])

rdd1.join(rdd2).collect()

[('foo', (1, 4)), ('bar', (2, 5)), ('bar', (2, 6))]

In [26]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 4)), ('bar', (2, 5)), ('bar', (2, 6)), ('baz', (3, None))]

In [27]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 4)), ('bar', (2, 5)), ('bar', (2, 6)), ('zoo', (None, 1))]

# Summary
- Key-Value (Pairs) RDD Operations
    - groupBykey
    - rdeuceByKey
    - mapValues
    - keys
    - countByValues
- Joins 
    - Inner join
    - Outer Join 
- 배운것 외에 여러 Operation이 존재

In [32]:
list1 = [1,2,3,4,5,6]
list2 = [4,5,6,7,8,9]

def find_common_elements(c1, c2):
    result = list(set(c1).intersection(c2))
    return result

find_common_elements(list1, list2)
    

[4, 5, 6]

In [34]:
def find_common_elements(list1, list2):
    return set(list1) & set(list2)

list1 = [1, 2, 3, 4, 5, 6]
list2 = [4, 5, 6, 7, 8, 9]
result = find_common_elements(list1, list2)
result

{4, 5, 6}

In [38]:
set(list1)&set(list2)

{4, 5, 6}